In [ ]:
from google.colab import drive
import ee
import geemap
import pandas as pd
import numpy as np
import random
import os
import cv2 as cv


drive.mount('/content/drive')

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='andres-gara2024')

In [ ]:
def SAR_Imageries_Mean(Start, End, Latitude, Longitude, Polarization, Direction = 'DESCENDING', Degrees = 0.1):

  Degrees2 = Degrees/2
  p11 = ee.Geometry.Point(Longitude, Latitude)
  p12 = ee.Geometry.Point(Longitude, Latitude + Degrees)
  p21 = ee.Geometry.Point(Longitude  + Degrees2, Latitude)
  p22 = ee.Geometry.Point(Longitude + Degrees2, Latitude + Degrees)

  SAR = (ee.ImageCollection('COPERNICUS/S1_GRD').
        filter(ee.Filter.listContains('transmitterReceiverPolarisation', Polarization)).
        filterBounds(p11).filterBounds(p12).filterBounds(p21).filterBounds(p22).filterDate(Start, End).
        filter(ee.Filter.eq('instrumentMode', 'IW')).
        filter(ee.Filter.eq('orbitProperties_pass', Direction)).
        select(Polarization))

  ROI = ee.Geometry.Rectangle(Longitude, Latitude, Longitude + Degrees, Latitude + Degrees/2)
  SAR_roi = SAR.mean().clip(ROI)

  return SAR_roi

def Optics_Imageries_Mean(Start, End, Latitude, Longitude, Cloudiness = 5, Water_percentage = 40, Snow_percentage = 5, Vegetation_percentage = 10, Degrees = 0.1):

  Degrees2 = Degrees/2
  p11 = ee.Geometry.Point(Longitude, Latitude)
  p12 = ee.Geometry.Point(Longitude, Latitude + Degrees)
  p21 = ee.Geometry.Point(Longitude  + Degrees2, Latitude)
  p22 = ee.Geometry.Point(Longitude + Degrees2, Latitude + Degrees)

  Optics = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').
         filterDate(Start, End).
         filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', Cloudiness)).
         filter(ee.Filter.lt('SNOW_ICE_PERCENTAGE', Snow_percentage)).
         filter(ee.Filter.lt('WATER_PERCENTAGE', Water_percentage)).
         filter(ee.Filter.gt('VEGETATION_PERCENTAGE', Vegetation_percentage)).
         filterBounds(p11).filterBounds(p12).filterBounds(p21).filterBounds(p22))

  ROI = ee.Geometry.Rectangle(Longitude, Latitude, Longitude + Degrees, Latitude + Degrees/2)
  Optics_roi = Optics.mean().clip(ROI)
  Size_value = Optics.size().getInfo()

  if (Size_value > 0):

    Optics_list = Optics.toList(Optics.size())

    Cloudiness_Volatil = []
    Water_percentage_Volatil = []
    Vegetation_percentage_Volatil = []


    for i in range(Optics_list.size().getInfo()):

        IMG = Optics.toList(Optics_list.size()).get(i)
        IMG = ee.Image(IMG)
        IMG = IMG.clip(ROI)

        propiedades = IMG.getInfo()
        Cloudiness_Value = propiedades['properties']['CLOUD_COVERAGE_ASSESSMENT']
        Water_percentage_Value = propiedades['properties']['WATER_PERCENTAGE']
        Vegetation_percentage_Value = propiedades['properties']['VEGETATION_PERCENTAGE']

        Cloudiness_Volatil.append(Cloudiness_Value)
        Water_percentage_Volatil.append(Water_percentage_Value)
        Vegetation_percentage_Volatil.append(Vegetation_percentage_Value)

    Cloudiness_Mean_Volatil = sum(Cloudiness_Volatil) / len(Cloudiness_Volatil)
    Water_percentage_Mean_Volatil = sum(Water_percentage_Volatil) / len(Water_percentage_Volatil)
    Vegetation_percentage_Mean_Volatil = sum(Vegetation_percentage_Volatil) / len(Vegetation_percentage_Volatil)

    Optics_roi = Optics.mean().clip(ROI)
    return  Size_value, Optics_roi, Cloudiness_Mean_Volatil, Water_percentage_Mean_Volatil, Vegetation_percentage_Mean_Volatil
  if (Size_value == 0):
    return  Size_value, Optics_roi, 0, 0, 0

In [ ]:
folderSAR = '/content/drive/MyDrive/Main_Folder/Original/SAR/'
folderNDVI = '/content/drive/MyDrive/Main_Folder/Original/NDVI/'
folderOptics = '/content/drive/MyDrive/Main_Folder/Original/Optics/'
Metadata = '/content/drive/MyDrive/Main_Folder/Metadata/'


os.makedirs(folderSAR + 'VV', exist_ok=True)
os.makedirs(folderSAR + 'VH', exist_ok=True)
os.makedirs(folderNDVI, exist_ok=True)
os.makedirs(folderOptics, exist_ok=True)
os.makedirs(Metadata, exist_ok=True)

In [ ]:
i = 0
Number_images = 1000

Initial_dates = []
End_dates = []
Latitudes = []
Longitudes = []
Cloudiness_Mean = []
Water_percentage_Mean = []
Vegetation_percentage_Mean = []

while i < Number_images:

  Latitude = random.uniform(-40, 40)
  Longitude = random.uniform(-179, 179)
  Day = random.randint(1, 28)
  Month = random.randint(1, 9)
  Year = random.randint(17, 23)

  Start_date = '20' + str(Year) + '-' + str(Month) + '-' + str(Day)
  End_date = '20' + str(Year) + '-' + str(Month + 3) + '-' + str(Day)

  sarVV = SAR_Imageries_Mean(Start_date, End_date, Latitude, Longitude, Polarization = 'VV')
  sarVH = SAR_Imageries_Mean(Start_date, End_date, Latitude, Longitude, Polarization = 'VH')
  Size_value, Optics, Cloudiness_Mean_Volatil, Water_percentage_Mean_Volatil, Vegetation_percentage_Mean_Volatil = Optics_Imageries_Mean(Start_date, End_date, Latitude, Longitude)

  if (Size_value > 0):
    Size_value, Optics, Cloudiness_Mean_Volatil, Water_percentage_Mean_Volatil, Vegetation_percentage_Mean_Volatil = Optics_Imageries_Mean(Start_date, End_date, Latitude, Longitude)
    CONDITION_1 = Cloudiness_Mean_Volatil < 5
    CONDITION_2 = Water_percentage_Mean_Volatil < 40
    CONDITION_3 = Vegetation_percentage_Mean_Volatil > 10

    if ((len(sarVV.getInfo()['bands'])) and len(Optics.getInfo()['bands']) and CONDITION_1 and CONDITION_2 and CONDITION_3):

      NDVI = Optics.normalizedDifference(['B8', 'B4'])
      Optics_vis = Optics.visualize(**{'min' : 0.0, 'max' : 3000, 'bands': ['B4', 'B3', 'B2']})
      sarVV_vis = sarVV.visualize(**{'min' : -25, 'max' : 5})
      sarVH_vis = sarVH.visualize(**{'min' : -25, 'max' : 5})

      geemap.ee_export_image(NDVI, filename = folderNDVI + str(i) +'.tif', scale = 10, file_per_band=False)
      geemap.ee_export_image(Optics_vis, filename = folderOptics + str(i) +'.tif', scale = 10, file_per_band=False)
      geemap.ee_export_image(sarVV_vis, filename = folderSAR + 'VV/' + str(i) +'.tif', scale = 10, file_per_band=False)
      geemap.ee_export_image(sarVH_vis, filename = folderSAR + 'VH/' + str(i) +'.tif', scale = 10, file_per_band=False)

      Initial_dates.append(Start_date)
      End_dates.append(End_date)
      Latitudes.append(Latitude)
      Longitudes.append(Longitude)

      Cloudiness_Mean.append(Cloudiness_Mean_Volatil)
      Water_percentage_Mean.append(Water_percentage_Mean_Volatil)
      Vegetation_percentage_Mean.append(Vegetation_percentage_Mean_Volatil)

      print('Imagary #(',i,') save successfully!')
      i += 1

In [ ]:
Metadata_xlsx = '/content/drive/MyDrive/Main_Folder/Metadata/Data.xlsx'
Metadata_csv = '/content/drive/MyDrive/Main_Folder/Metadata/Data.csv'

df = pd.DataFrame(data={'Initial dates': Initial_dates,
                        'End dates': End_dates,
                        'Latitud': Latitudes,
                        'Longitudes': Longitudes,
                        'Cloudiness Mean': Cloudiness_Mean,
                        'Water percentage Mean': Water_percentage_Mean,
                        'Vegetation percentage Mean': Vegetation_percentage_Mean})
df.to_csv(Metadata_csv)
df.to_excel(Metadata_xlsx)

In [ ]:
input_images_path = '/content/drive/MyDrive/Main_Folder/Original/'
output_images_path = '/content/drive/MyDrive/Main_Folder/Crop/'
paths = ['SAR/VH/', 'Optics/', 'NDVI/', 'SAR/VV/']

for path in paths:
    files_names = os.listdir(input_images_path + path)
    os.makedirs(output_images_path + path, exist_ok=True)
    for file_name in files_names:
        image_path = input_images_path + path +  file_name

        img2 = cv.imread(image_path, cv.IMREAD_UNCHANGED)
        img3 = img2[:512, :512]
        img4 = img2[:512, 512:1024]
        split = file_name.split('.')
        #split[1] --> 'tiff'
        #split[0] --> 'image's name'
        cv.imwrite(output_images_path + path + split[0] + 'A.' + split[1], img3)
        cv.imwrite(output_images_path + path + split[0] + 'B.' + split[1], img4)
    print('Finished', path ,'folder')